# Demo de Análisis de Abandono de Clientes
En este notebook veremos primero como acceder a datos que están realmente alojados en AWS, en formato CSV, para cargarlos en un dataframe de pandas, limpiar y cruzar esos datos, y posteriormente entrenar un modelo de clasificación para detectar ese abandono. Veremos como utilizar Data Wrangler para analizar esos datos y poder limpiarlos, así como la integración con MlFlow para gestionar el ciclo de experimento --> modelo

In [1]:
#Instalamos las librerías necesarias

#!pip install imblearn 

#NO lo necesitamos, está en el entorno :-)

StatementMeta(, 41ed7201-40ff-48f4-9477-56564f25423e, 3, Finished, Available)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.6/235.6 kB 22.4 MB/s eta 0:00:00


In [1]:
#importamos las librerías necesarias
import pandas as pd
import numpy as np

StatementMeta(, 60663dd2-4838-4f06-8734-3ebd56f670e8, 5, Finished, Available)

In [4]:
#Cargamnos con pandas los datos que están en AWS a través del enlace 
df_transactions = pd.read_csv("/lakehouse/default/Files/dunnhumby - The Complete Journey CSV/transaction_data.csv")

StatementMeta(, 60663dd2-4838-4f06-8734-3ebd56f670e8, 8, Finished, Available)

In [5]:
#Calculamos el ticket medio de compra

df_basket= df_transactions.groupby(by="BASKET_ID").agg({'SALES_VALUE':'sum'}).sort_values(by='BASKET_ID',ascending=False)
Average_ticket = df_basket.sum() / df_basket.count()

StatementMeta(, 60663dd2-4838-4f06-8734-3ebd56f670e8, 9, Finished, Available)

In [6]:
#Agrupando por tienda, vemos que hay tiendas con solo una venta, por ejemplo

df_basket_store= df_transactions.groupby(by="STORE_ID").agg({'SALES_VALUE':'sum' , 'BASKET_ID':'count'}).sort_values(by='SALES_VALUE',ascending=False)
df_basket_store['Average_Ticket'] = df_basket_store['SALES_VALUE'] / df_basket_store['BASKET_ID']
df_basket_store.sort_values(by=["Average_Ticket"], ascending=False)

StatementMeta(, 60663dd2-4838-4f06-8734-3ebd56f670e8, 10, Finished, Available)

,SALES_VALUE,BASKET_ID,Average_Ticket
STORE_ID,,,
3065,72.00,1,72.000000
489,51.60,1,51.600000
86,50.82,1,50.820000
3098,43.62,1,43.620000
1162,41.00,1,41.000000
...,...,...,...
2825,1.54,3,0.513333
751,3.59,7,0.512857
610,0.50,1,0.500000


In [7]:
#Agrupamos por clientes, y vemos que tenemos clientes muy fieles

df_sales_bycustomer = df_transactions.groupby(by="household_key").agg({'SALES_VALUE':'sum', 'BASKET_ID':'count'}).sort_values(by='BASKET_ID',ascending=False)
df_sales_bycustomer

StatementMeta(, 60663dd2-4838-4f06-8734-3ebd56f670e8, 11, Finished, Available)

,SALES_VALUE,BASKET_ID
household_key,,
718,19299.86,6851
2459,20671.50,6646
1609,27859.68,6625
1111,18894.72,6576
1453,21661.29,6561
...,...,...
1626,42.96,8
1504,66.33,7
1830,39.44,7


In [8]:
df_sales_bycustomer['Average_Ticket'] = df_sales_bycustomer['SALES_VALUE'] / df_sales_bycustomer['BASKET_ID']
df_sales_bycustomer.sort_values(by=["BASKET_ID"], ascending=False).head(10)

StatementMeta(, 60663dd2-4838-4f06-8734-3ebd56f670e8, 12, Finished, Available)

,SALES_VALUE,BASKET_ID,Average_Ticket
household_key,,,
718,19299.86,6851,2.817087
2459,20671.50,6646,3.110367
1609,27859.68,6625,4.205235
1111,18894.72,6576,2.873285
1453,21661.29,6561,3.301523
982,18790.34,5806,3.236366
2322,23646.92,5692,4.154413
371,15716.54,5669,2.772365
1489,17251.53,5586,3.088351


In [9]:
# Código generado por Data Wrangler para pandas DataFrame

def clean_data(df_transactions):
    # Se ha realizado 1 agregación agrupada en la columna: 'STORE_ID'
    df_transactions = df_transactions.groupby(['STORE_ID']).agg(SALES_VALUE_sum=('SALES_VALUE', 'sum')).reset_index()
    return df_transactions

df_transactions_clean = clean_data(df_transactions.copy())
df_transactions_clean.head()

StatementMeta(, 60663dd2-4838-4f06-8734-3ebd56f670e8, 13, Finished, Available)

,STORE_ID,SALES_VALUE_sum
0,1,29.89
1,2,22.06
2,12,12.06
3,19,28.49
4,20,20.00


In [10]:
#Hacemos un cálculo rápido de Recencia, Frecuencia y valor Monetario de nuestros clientes. Aquellos clientes que hace más de 600 días que no nos compran los consideramos perdidos

df_RFM = df_transactions.groupby(by="household_key").agg({'SALES_VALUE':'sum' , 'BASKET_ID':'nunique', 'DAY':'max'}).sort_values(by='SALES_VALUE',ascending=False)

df_RFM.rename(columns={'DAY': 'Recency',
                         'BASKET_ID': 'Frequency',
                         'SALES_VALUE': 'MonetaryValue'}, inplace=True)



df_lostcustomers = df_RFM[df_RFM['Recency'] < 600]
df_lostcustomers.head()

StatementMeta(, 60663dd2-4838-4f06-8734-3ebd56f670e8, 14, Finished, Available)

,MonetaryValue,Frequency,Recency
household_key,,,
1828,5813.77,86,570
1974,5266.00,198,510
2051,4923.16,288,462
682,4438.29,217,569
1031,4232.12,146,557


In [12]:
#Cargamos los datos demográficos de los clientes

df_customers = pd.read_csv("/lakehouse/default/Files/dunnhumby - The Complete Journey CSV/hh_demographic.csv")

StatementMeta(, 60663dd2-4838-4f06-8734-3ebd56f670e8, 16, Finished, Available)

In [13]:
#Nos quedamos con todos los clientes que haya comprado alguna vez, y con sus datos demográficos

join_customers = df_customers.join(df_lostcustomers , on='household_key' , how='left')

join_customers['Churn'] = np.isnan(join_customers['MonetaryValue']) #True are customer who stay

features =['household_key','AGE_DESC' , 'MARITAL_STATUS_CODE' , 'INCOME_DESC' , 'HOMEOWNER_DESC' , 'HH_COMP_DESC' , 'HOUSEHOLD_SIZE_DESC' , 'KID_CATEGORY_DESC']
label = ['Churn']

StatementMeta(, 60663dd2-4838-4f06-8734-3ebd56f670e8, 17, Finished, Available)

In [14]:
df_churn = join_customers[features]
y= join_customers["Churn"]

StatementMeta(, 60663dd2-4838-4f06-8734-3ebd56f670e8, 18, Finished, Available)

In [15]:
# Codificamos las variables categóricas
dum_df = pd.get_dummies(df_churn, columns=['AGE_DESC' , 'MARITAL_STATUS_CODE' , 'INCOME_DESC' , 'HOMEOWNER_DESC' , 'HH_COMP_DESC' , 'HOUSEHOLD_SIZE_DESC' , 'KID_CATEGORY_DESC'] )

#Escalamos las numéricas

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(dum_df)


StatementMeta(, 60663dd2-4838-4f06-8734-3ebd56f670e8, 19, Finished, Available)

In [16]:
#Balanceamos nuestra etiqueta
#Se crea un experimento de forma automática?

from imblearn.over_sampling import SMOTE 
from collections import Counter
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X, y)

StatementMeta(, 60663dd2-4838-4f06-8734-3ebd56f670e8, 20, Finished, Available)

2024-06-15:07:36:19,959 WARNING  [tracking_store.py:153] log_inputs not supported


In [17]:
# ahora podemos crear el experimento basándonos en este modelo entrenado
import mlflow
mlflow.create_experiment("churn-dunnhumby")
mlflow.set_experiment("churn-dunnhumby")

StatementMeta(, 60663dd2-4838-4f06-8734-3ebd56f670e8, 21, Finished, Available)

<Experiment: artifact_location='', creation_time=1718437079017, experiment_id='015485de-bd8f-4b07-94c0-bf4bf1772e41', last_update_time=None, lifecycle_stage='active', name='churn-dunnhumby', tags={}>

In [18]:
#Abordamos el problema de clasificación, con un algoritmo de KNN Clasificador

with mlflow.start_run() as run:

     from sklearn.neighbors import KNeighborsClassifier
     from sklearn.model_selection import train_test_split
     X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, random_state=0)
     knn = KNeighborsClassifier()
     knn.fit(X_train, y_train)
     
     #predicción
     pred = knn.predict(X_train)
     
     #Obtenemos métricas
     score_training = knn.score(X_train, y_train)
     score_test =  knn.score(X_test, y_test)
     
     #Registramos las métricas de este entrenamiento
     mlflow.log_metrics({"score_training" : score_training , "score_test" : score_test})
     
     #Registramos el modelo en el registro de mlflow
     mlflow.sklearn.log_model(knn , "modelo-churn-dunnhumby")
     
     #Creamos el objeto modelo en Fabric para reutilizar
     mlflow.register_model("runs:/{}/modelo-churn-dunnhumby".format(run.info.run_id) , "demo-churn-dunhumby")
 

StatementMeta(, 60663dd2-4838-4f06-8734-3ebd56f670e8, 22, Finished, Available)

Successfully registered model 'demo-churn-dunhumby'.
2024/06/15 07:38:18 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: demo-churn-dunhumby, version 1
Created version '1' of model 'demo-churn-dunhumby'.


In [19]:
#Analizamos la matriz de confusión y el informe de clasificación

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

print(confusion_matrix(y_train, pred))
print(classification_report(y_train, pred))

StatementMeta(, 60663dd2-4838-4f06-8734-3ebd56f670e8, 23, Finished, Available)

[[606   0]
 [ 28 558]]
              precision    recall  f1-score   support

       False       0.96      1.00      0.98       606
        True       1.00      0.95      0.98       586

    accuracy                           0.98      1192
   macro avg       0.98      0.98      0.98      1192
weighted avg       0.98      0.98      0.98      1192

